# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units=8, activation='tanh'))

### Adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 150)

Epoch 1/100
250/250 [==============================] - 4s 9ms/step - loss: 0.5606 - accuracy: 0.7883
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4792 - accuracy: 0.8005
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4621 - accuracy: 0.7892
Epoch 4/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4497 - accuracy: 0.7879
Epoch 5/100
250/250 [==============================] - 3s 10ms/step - loss: 0.4247 - accuracy: 0.7967
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 0.4116 - accuracy: 0.8106
Epoch 7/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3862 - accuracy: 0.8198
Epoch 8/100
250/250 [==============================] - 2s 7ms/step - loss: 0.4087 - accuracy: 0.8037
Epoch 9/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3884 - accuracy: 0.8246
Epoch 10/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3878 - accur

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [21]:
y_pred = ann.predict(X_test)
y_pred

array([[0.24702907],
       [0.32518077],
       [0.18671393],
       ...,
       [0.21822709],
       [0.22503996],
       [0.18493804]], dtype=float32)

In [22]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1513   82]
 [ 194  211]]


---
# Avaliando o Modelo

In [24]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [25]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [26]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [27]:
print(mean,variance)

0.8317499935626984 0.010018729741884213


# Melhorando o modelo

In [33]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [34]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [35]:
print(mean,variance)

0.8318750023841858 0.011716037970497422


# Tuning the ANN

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [ ]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 5s 5ms/step - loss: 0.5624 - accuracy: 0.7926
Epoch 2/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4308 - accuracy: 0.7996
Epoch 3/50
720/720 [==============================] - 5s 6ms/step - loss: 0.4358 - accuracy: 0.7971
Epoch 4/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4300 - accuracy: 0.7975
Epoch 5/50
720/720 [==============================] - 4s 5ms/step - loss: 0.4335 - accuracy: 0.8171
Epoch 6/50
720/720 [==============================] - 3s 4ms/step - loss: 0.4271 - accuracy: 0.8199
Epoch 7/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4203 - accuracy: 0.8276
Epoch 8/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4196 - accuracy: 0.8277
Epoch 9/50
720/720 [==============================] - 6s 8ms/step - loss: 0.4011 - accuracy: 0.8354
Epoch 10/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4069 - accuracy: 0.8378

/home/mateus/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/50
720/720 [==============================] - 6s 7ms/step - loss: 0.5662 - accuracy: 0.7900
Epoch 2/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4324 - accuracy: 0.7992
Epoch 3/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4322 - accuracy: 0.8015
Epoch 4/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4207 - accuracy: 0.8291
Epoch 5/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4331 - accuracy: 0.8229
Epoch 6/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4200 - accuracy: 0.8263
Epoch 7/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4103 - accuracy: 0.8343
Epoch 8/50
720/720 [==============================] - 6s 8ms/step - loss: 0.4092 - accuracy: 0.8321
Epoch 9/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4206 - accuracy: 0.8280
Epoch 10/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4124 - accuracy: 0.8353

/home/mateus/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/50
720/720 [==============================] - 6s 7ms/step - loss: 0.5725 - accuracy: 0.7918
Epoch 2/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4477 - accuracy: 0.7912
Epoch 3/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4245 - accuracy: 0.7996
Epoch 4/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4048 - accuracy: 0.8303
Epoch 5/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4081 - accuracy: 0.8364
Epoch 6/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4128 - accuracy: 0.8331
Epoch 7/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4168 - accuracy: 0.8338
Epoch 8/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4021 - accuracy: 0.8426
Epoch 9/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4250 - accuracy: 0.8249
Epoch 10/50
720/720 [==============================] - 4s 5ms/step - loss: 0.4182 - accuracy: 0.8334

...

Epoch 1/50
720/720 [==============================] - 3s 3ms/step - loss: 0.5614 - accuracy: 0.7928
Epoch 2/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4338 - accuracy: 0.7928
Epoch 3/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4290 - accuracy: 0.8038
Epoch 4/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4175 - accuracy: 0.8310
Epoch 5/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4251 - accuracy: 0.8262
Epoch 6/50
720/720 [==============================] - 4s 5ms/step - loss: 0.4238 - accuracy: 0.8271
Epoch 7/50
720/720 [==============================] - 4s 5ms/step - loss: 0.4031 - accuracy: 0.8367
Epoch 8/50
720/720 [==============================] - 5s 6ms/step - loss: 0.4096 - accuracy: 0.8349
Epoch 9/50
720/720 [==============================] - 4s 6ms/step - loss: 0.4032 - accuracy: 0.8294
Epoch 10/50
720/720 [==============================] - 4s 5ms/step - loss: 0.4095 - accuracy: 0.8280

/home/mateus/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/50
720/720 [==============================] - 6s 7ms/step - loss: 0.5628 - accuracy: 0.7906
Epoch 2/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4424 - accuracy: 0.7922
Epoch 3/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4270 - accuracy: 0.8003
Epoch 4/50
720/720 [==============================] - 6s 8ms/step - loss: 0.4155 - accuracy: 0.8301
Epoch 5/50
720/720 [==============================] - 6s 8ms/step - loss: 0.4301 - accuracy: 0.8219
Epoch 6/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4192 - accuracy: 0.8262
Epoch 7/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4160 - accuracy: 0.8276
Epoch 8/50
720/720 [==============================] - 5s 7ms/step - loss: 0.4222 - accuracy: 0.8250
Epoch 9/50
720/720 [==============================] - 5s 6ms/step - loss: 0.4258 - accuracy: 0.8231
Epoch 10/50
720/720 [==============================] - 6s 8ms/step - loss: 0.4157 - accuracy: 0.8225

/home/mateus/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Epoch 1/50
720/720 [==============================] - 6s 7ms/step - loss: 0.5778 - accuracy: 0.7928
Epoch 2/50
 96/720 [===>..........................] - ETA: 4s - loss: 0.4343 - accuracy: 0.7963

In [ ]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

# Halving Grid Search

In [36]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100, 150],
    'optimizer': ['adam', 'rmsprop', 'SGD']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="balanced_accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

In [ ]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

/home/mateus/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)